# Setting up a Ray cluster with SmartSim

## 1. Start the cluster
We set up a SmartSim experiment, which will handle the launch of the Ray cluster.

First we import the relevant modules.

In [5]:
from smartsim import Experiment, slurm
from smartsim.ray import RayCluster

NUM_WORKERS = 5
alloc=None

In [6]:
exp = Experiment("ray-cluster", launcher='slurm')

cluster = RayCluster(name="ray-cluster", path='', launcher='slurm', workers=NUM_WORKERS, alloc=alloc, batch=True)
exp.generate(cluster, overwrite=True)

exp.start(cluster, block=False, summary=False)

12:33:29 osprey.us.cray.com SmartSim[35303] INFO Working in previously created experiment
12:33:33 osprey.us.cray.com SmartSim[35303] INFO Ray cluster launched on nodes: ['prod-0001', 'prod-0003', 'prod-0006', 'prod-0005', 'prod-0004', 'prod-0002']


## 2. Start the ray driver script

In [7]:
cluster.start_ray_job('/lus/sonexion/arigazzi/smartsim-dev/SmartSim/tutorials/05_starting_ray/templates/ppo_tune.py')

## 3. Stop cluster and release allocation

In [8]:
exp.stop(cluster)

12:39:34 osprey.us.cray.com SmartSim[35303] INFO Stopping model workers with job name workers-CB76ZFTDRBUL
12:39:46 osprey.us.cray.com SmartSim[35303] INFO Stopping model head with job name head-CB76ZEDP49HS


In [9]:
# slurm.release_allocation(alloc)

05:51:53 osprey.us.cray.com SmartSim[131434] INFO Releasing allocation: 230194
05:51:53 osprey.us.cray.com SmartSim[131434] INFO Successfully freed allocation 230194


In [4]:
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            231323     bdw18 Chpl-isx  chapelu  R      24:50     16 prod-[0015-0030]
            231324     bdw18 interact  builder  R      19:11      8 prod-[0031-0038]
            231327     bdw18 head-CB7 arigazzi  R       1:47      1 prod-0001
            231328     bdw18 workers- arigazzi  R       1:44      5 prod-[0002-0006]
